In [44]:
import matplotlib.pyplot as plt
import numpy as np
from lteanalysis import LTEAnalysis
from matplotlib import cm

In [2]:
%matplotlib widget

In [3]:
def cost_function(T, N, X, Y, model):

    # print("printing params", params)
    N = N

    T = T

    X_predicted = model.get_intensity(line = 'c18o', Ju = 3, Ncol = N, Tex = T, delv = 0.5, Xconv = 1.e-7) # band 6
    Y_predicted = model.get_intensity(line = 'c18o', Ju = 2, Ncol = N, Tex = T, delv = 0.5, Xconv = 1.e-7) # band 7

    #print(f"For T = {T}, N = {N:.2e}")
    #print(f"Tb6 = {X_predicted}, Tb7 = {Y_predicted}")


    error = (X_predicted - X)**2 + (Y_predicted - Y)**2

    #print(np.shape(error))
    return error

In [28]:
T = np.linspace(10, 50, 1000)
lg_N = np.linspace(5,20,1000)

tt, n = np.meshgrid(T,lg_N)
#print(T)
#print(10**(lg_N))

In [29]:
# ----- input -----
# for LTE calculation
line  = 'c18o'
Xconv = 1e-7
delv  = 0.5 # km/s
ilines = [3,2] # Ju
Ncols = np.array([5.e16, 5.9e16, 5.e17]) # cm^-2  
Texes = np.array([5, 18, 22, 30]) # K

# Initiate Model

lte_model = LTEAnalysis()
lte_model.read_lamda_moldata(line)

Z = np.zeros((len(T), len(lg_N)))


for i in range(len(T)):

    for j in range(len(lg_N)):


        X_predicted = lte_model.get_intensity(line = 'c18o', Ju = 3, Ncol = 10**lg_N[j], Tex = T[i], delv = 0.5, Xconv = 1.e-7) # band 6
        Y_predicted = lte_model.get_intensity(line = 'c18o', Ju = 2, Ncol = 10**lg_N[j], Tex = T[i], delv = 0.5, Xconv = 1.e-7) # band 7

        # print(f"For T = {T[i]}, N = {10**(lg_N[j]):.2e}")
        # print(f"Tb6 = {X_predicted}, Tb7 = {Y_predicted}")

        Z[i, j] = cost_function(T[i], 10**(lg_N[j]), X = 10.1, Y = 10., model = lte_model)


In [40]:
np.where(Z == np.min(Z))

print(T[217])
print(lg_N[848])

print(lte_model.get_intensity(line = 'c18o', Ju = 3, Ncol = 10**lg_N[848], Tex = T[217], delv = 0.5, Xconv = 1.e-7))
print(lte_model.get_intensity(line = 'c18o', Ju = 2, Ncol = 10**lg_N[848], Tex = T[217], delv = 0.5, Xconv = 1.e-7))

18.68868868868869
17.73273273273273
10.10093874312323
10.002497941115617


In [ ]:
fig = plt.figure(figsize = (10,10))
ax = plt.axes(projection='3d')
cp = ax.plot_surface(n, tt, Z, cmap=cm.coolwarm, linewidth=10, antialiased=False)
ax.set_xlabel('t')
ax.set_ylabel('N')
ax.set_zlabel('z')
ax.set_zlim(-100,1000)
fig.colorbar(cp, shrink=0.5, aspect=5)
plt.show()